# Análise de Avaliações de Filmes

Este notebook conecta ao Data Lake (MinIO), lê os dados processados (Camada Silver) e gera visualizações.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import s3fs

# Configuração do MinIO
minio_endpoint = "http://minio:9000"
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': minio_endpoint},
    key='minioadmin',
    secret='minioadmin',
    use_listings_cache=False
)

## 1. Top 10 Filmes por Nota Média

In [ ]:
# Ler dados do bucket Silver
with fs.open("s3://silver/movie_ratings", "rb") as f:
    # Nota: Parquet particionado pode precisar de leitura via pyarrow.parquet.ParquetDataset
    # Para simplificar aqui, assumimos que pandas consegue ler o diretório via s3fs se passarmos o path correto
    # Ou usamos pyarrow direto.
    df_ratings = pd.read_parquet("s3://silver/movie_ratings", filesystem=fs)

top10 = df_ratings.sort_values("avg_rating", ascending=False).head(10)

plt.figure(figsize=(10,6))
plt.barh(top10["title"], top10["avg_rating"], color='skyblue')
plt.xlabel("Avaliação Média")
plt.title("Top 10 Filmes")
plt.gca().invert_yaxis()
plt.show()

## 2. Distribuição de Gêneros

In [ ]:
df_genres = pd.read_parquet("s3://silver/genre_counts", filesystem=fs)
df_genres = df_genres.sort_values("count", ascending=False).head(10)

plt.figure(figsize=(8,8))
plt.pie(df_genres["count"], labels=df_genres["genre"], autopct='%1.1f%%', startangle=140)
plt.title("Top 10 Gêneros Mais Avaliados")
plt.show()